In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'january'
new_month = 'february'

In [3]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='coinmarketcap',index_col='Ticker')

In [4]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [66]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.loc['IOTA'] = df.loc['IOT']
df.loc['VEN'] = df.loc['VET']
df.index = map(lambda x: x.lower(), df.index)

In [67]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['circulating'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply']+0.0001)
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [68]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

In [69]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [70]:
df.loc['IOT'] = df.loc['IOTA']
df.loc['VET'] = df.loc['VEN']

In [71]:
#read in other data sources
onchainfx_db = df

In [72]:
#remove duplicates
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

988
135
135
988
135
135


In [73]:
#fix any ticker mapping issues
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})

In [74]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [75]:
#main function to create table of all assets
def create_universe(onchainfx,coinmarketcap,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    for t in eligibility.index:
        if t in coinmarketcap.index:
            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
        else:
            print (t,' no volume data!')
            eligibility.loc[t,('Volume')] = 1
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [76]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [77]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,coinmarketcap_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

CENNZ  no volume data!
ECN  no volume data!
MCAP  no volume data!
PLR  no volume data!
POT  no volume data!
IOTA  no volume data!
VEN  no volume data!
duplicate tickers


In [78]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
BTM,1.650810e+08,7.880649e+07,4.773806e-01,1.002499e+09,0.078610,2.100000e+09,1,82018572.0,1.040759,1,0.0,2.0,0.0,2.0,0
CTXC,3.153713e+07,1.575765e+07,4.996539e-01,1.497925e+08,0.105197,2.997925e+08,1,67940143.0,4.311566,1,0.0,2.0,0.0,2.0,0
DENT,1.168999e+08,2.015517e+07,1.724139e-01,1.724139e+10,0.001169,1.000000e+11,1,41365391.0,2.052347,1,0.0,3.0,0.0,2.0,0
DRGN,3.373801e+07,1.855567e+07,5.499929e-01,2.384219e+08,0.077827,4.335000e+08,1,14801065.0,0.797657,1,0.0,3.0,0.0,2.0,0
ETN,0.000000e+00,6.062470e+07,8.821605e+13,8.821605e+09,0.006872,0.000000e+00,1,7567002.0,0.124817,1,0.0,3.0,0.0,2.0,0
R,1.954135e+08,9.466806e+07,4.844500e-01,4.844500e+08,0.195413,1.000000e+09,1,88930338.0,0.939391,1,0.0,3.0,0.0,2.0,0
RHOC,2.349024e+07,1.019573e+07,4.340412e-01,3.737899e+08,0.027277,8.611852e+08,1,4871189.0,0.477767,1,0.0,3.0,0.0,2.0,0
SMART,4.959390e+07,1.045214e+07,2.107546e-01,1.053773e+09,0.009919,5.000000e+09,1,6161890.0,0.589534,1,0.0,3.0,0.0,2.0,0
SPANK,1.286741e+07,3.860781e+06,3.000434e-01,3.000434e+08,0.012867,1.000000e+09,1,802387.0,0.207830,1,0.0,3.0,1.0,2.0,0


In [79]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
universe = universe.rename({'HSR' : 'HC'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Float Pass,Volume,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,7.433239e+10,6.200956e+10,0.834220,1.750485e+07,3542.421705,2.098350e+07,1,1.512465e+11,2.439084,1,1.0,1.0,0.0,3.0,1,1
XRP,3.061007e+10,6.679477e+09,0.218212,2.182117e+10,0.306101,1.000000e+11,1,1.308726e+10,1.959324,1,1.0,2.0,0.0,3.0,1,2
ETH,1.661058e+10,1.182076e+10,0.711640,1.045939e+08,113.015723,1.469759e+08,1,7.796009e+10,6.595186,1,1.0,2.0,0.0,3.0,1,3
XLM,1.344060e+10,1.806029e+09,0.134371,1.916682e+10,0.094227,1.426409e+11,1,3.010436e+09,1.666881,1,1.0,1.0,0.0,3.0,1,4
EOS,3.448095e+09,2.140309e+09,0.620722,9.062451e+08,2.361733,1.459985e+09,1,2.146275e+10,10.027875,1,1.0,2.0,0.0,3.0,1,5
TRX,2.880158e+09,1.919939e+09,0.666609,6.666089e+10,0.028802,1.000000e+11,1,6.754916e+09,3.518298,1,1.0,3.0,1.0,3.0,1,6
LTC,2.729198e+09,1.958993e+09,0.717791,6.021078e+07,32.535584,8.388348e+07,1,1.615697e+10,8.247591,1,1.0,1.0,0.0,3.0,1,7
BCH,2.557476e+09,2.143848e+09,0.838267,1.758972e+07,121.880709,2.098343e+07,1,6.357705e+09,2.965557,1,1.0,1.0,0.0,3.0,1,8
ADA,1.843047e+09,1.061884e+09,0.576157,2.592707e+10,0.040957,4.500000e+10,1,8.172854e+08,0.769656,1,1.0,2.0,0.0,3.0,1,9
BSV,1.494366e+09,1.252602e+09,0.838217,1.758866e+07,71.216458,2.098343e+07,1,2.026408e+09,1.617758,1,1.0,1.0,0.0,3.0,1,10


In [81]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained ADA
Retained BSV
10 full! No room for DASH
10 full! Do not add  ZEC


In [82]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained MANA
Retained LSK
Retained ZRX
Retained BTG
20 full! No room for SC
20 full! Do not add  WAVES
20 full! Do not add  HC
20 full! Do not add  STEEM


In [83]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained POWR
Retained AION
Retained QASH
Retained PPT
Retained WAN
Retained GNT
Retained FCT
Retained STORJ
Retained NAS
Retained MONA
40 full! Do not add  MCO
40 full! Do not add  THETA
40 full! Do not add  PAY
40 full! Do not add  GXS
40 full! Do not add  ARDR
40 full! Do not add  ARK
40 full! Do not add  CVC
40 full! Do not add  BLOCK
40 full! Do not add  FUN
40 full! Do not add  BNT


In [84]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [85]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [86]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

10 adds ['TRX']
10 deletes ['DASH']
10 count members: 10

40 adds ['SC']
40 deletes ['LINK']
40 count members: 40

Total adds []
Total deletes []
Total count members: 70

ETH adds []
ETH deletes []
ETH count members: 23

20 adds ['DASH' 'LINK']
20 deletes ['SC' 'TRX']
20 count members: 20

Application adds []
Application deletes []
Application count members: 24

Platform adds []
Platform deletes []
Platform count members: 26

Currency adds []
Currency deletes []
Currency count members: 20

Sector count members: 70


In [87]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])